In [1]:
import sys
import os

current_dir = os.getcwd()
print("当前目录:", current_dir)
# 获取当前文件的父目录
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
print(parent_dir)
from GPT_SoVITS.module.models import SynthesizerTrn

当前目录: /workspaces/GPT-SoVITS2/playground
/workspaces/GPT-SoVITS2


In [2]:
import sentencepiece as spm
import torch
sp = spm.SentencePieceProcessor()
sp.load("./sentencepiece.bpe.model")
token = sp.encode("你好啊，这里是测试token embedding效果怎么样")
print(token)
# 增加batch size 维度
token = torch.tensor(token).unsqueeze(0)
print(token)

[5, 124083, 6146, 3, 24474, 353, 49124, 6447, 32, 5, 55719, 59724, 15446, 93984]
tensor([[     5, 124083,   6146,      3,  24474,    353,  49124,   6447,     32,
              5,  55719,  59724,  15446,  93984]])


In [3]:
token.shape[1]

14

In [6]:
import time

params = {
    # hps.data.filter_length // 2 + 1,
    #     hps.train.segment_size // hps.data.hop_length,
    "spec_channels": 2048 // 2 + 1,
    "segment_size": 20480 // 640,
    "inter_channels": 256,
    "hidden_channels": 256,
    "filter_channels": 1024,
    "n_heads": 4,
    "n_layers": 6,
    "kernel_size": 3,
    "p_dropout": 0.1,
    "resblock": "1",
    "resblock_kernel_sizes": [3, 7, 11],
    "resblock_dilation_sizes": [[1, 3, 5], [1, 3, 5], [1, 3, 5]],
    "upsample_rates": [10, 8, 2, 2, 2],
    # "upsample_rates": [8,8,2,2],
    "upsample_initial_channel": 512,
    "upsample_kernel_sizes": [16, 16, 8, 2, 2],
    # "upsample_kernel_sizes": [16,16,4,4],
    "semantic_frame_rate": "50hz",
    "freeze_quantizer": True,
}
net_g = SynthesizerTrn(**params)
print(net_g)

res = net_g.forward(
    ssl=torch.randn(1, 768, 100),
    y=torch.randn(1, 2048 // 2 + 1, 100),
    y_lengths=torch.tensor([100]),
    token=torch.tensor(token),
    token_lengths=torch.tensor([token.shape[1]]),
)

SynthesizerTrn(
  (enc_p): TextEncoder(
    (ssl_proj): Conv1d(768, 256, kernel_size=(1,), stride=(1,))
    (encoder_ssl): Encoder(
      (drop): Dropout(p=0.1, inplace=False)
      (attn_layers): ModuleList(
        (0-5): 6 x MultiHeadAttention(
          (conv_q): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
          (conv_k): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
          (conv_v): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
          (conv_o): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
          (drop): Dropout(p=0.1, inplace=False)
        )
      )
      (norm_layers_1): ModuleList(
        (0-5): 6 x LayerNorm()
      )
      (ffn_layers): ModuleList(
        (0-5): 6 x FFN(
          (conv_1): Conv1d(256, 1024, kernel_size=(3,), stride=(1,))
          (conv_2): Conv1d(1024, 256, kernel_size=(3,), stride=(1,))
          (drop): Dropout(p=0.1, inplace=False)
        )
      )
      (norm_layers_2): ModuleList(
        (0-5): 6 x LayerNorm()
      )
    )


/tmp/ipykernel_111154/3654712241.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  token=torch.tensor(token),


In [7]:
res[1]

tensor([33])

In [9]:
print("number of trainable parameters:", sum(p.numel() for p in net_g.parameters() if p.requires_grad))

number of trainable parameters: 128489352


In [13]:
start_time = time.time()
for _ in range(1):
    res = net_g.infer(
        ssl=torch.randn(1, 768, 50),
        y=torch.randn(1, 2048 // 2 + 1, 50),
        y_lengths=torch.tensor([50]),
        token=torch.tensor(token),
        token_lengths=torch.tensor([token.shape[1]]),
    )
    res = net_g.extract_latent(
        x=torch.randn(1, 768, 50),
    )
    res = net_g.decode(
        codes=res.transpose(-1, -2),
        token=torch.tensor(token),
        refer=torch.randn(1, 2048 // 2 + 1, 50),
    )
    res.shape
print(time.time() - start_time)

/tmp/ipykernel_104519/1100293123.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  token=torch.tensor(token),


0.41435980796813965


/tmp/ipykernel_104519/1100293123.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  token=torch.tensor(token),


In [14]:
res.shape

torch.Size([1, 1, 32000])

In [8]:
torch.save(net_g, "net_g.pth")

torch.Size([1, 1, 64000])